In [1]:
import warnings
import numpy as np
import pandas as pd
from sklearn.metrics import pairwise_distances, accuracy_score
from scipy.spatial.distance import cdist, pdist
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.model_selection import StratifiedKFold
from sklearn_lvq import GlvqModel
from scipy.special import softmax

import seaborn as sns
import matplotlib.pyplot as plt
# %matplotlib notebook

sns.set(font_scale=2)
sns.set_style('white')
sns.set_context('poster')

In [2]:
def bipolarize(Y):
    X = np.copy(Y)
    X[X > 0] = 1.0
    X[X < 0] = -1.0
    X[X == 0] = np.random.choice([-1.0, 1.0], size=len(X[X == 0]))
    return X

In [3]:
def centroids(X,label=None):
    if label is not None:
        cLabel = np.unique(label)
        c = np.zeros((len(cLabel), X.shape[1]))
        for i,l in enumerate(cLabel):
            c[i,:] = bipolarize(np.sum(X[label==l],axis=0))
    else:
        c = bipolarize(np.sum(X,axis=0)).reshape(1,-1)
        cLabel = [0]
    return cLabel, c.astype('int')

In [4]:
def classify(v,am,metric):
    d = cdist(v,am,metric)
    label = np.argmin(d,axis=1)
    return label

In [5]:
# select dataset and encoding type
dataName = 'allHV.npz'
emgHVType =  'hvRel'

allHV = np.load(dataName)

# extract data and labels based on gesture, trial, and position
hv = allHV[emgHVType]
gestLabel = allHV['gestLabel']
posLabel = allHV['posLabel']
trialLabel = allHV['trialLabel']

combGP, groupGP = np.unique(np.column_stack((gestLabel,posLabel)),axis=0,return_inverse=True)
combGPT, groupGPT = np.unique(np.column_stack((gestLabel,posLabel,trialLabel)),axis=0,return_inverse=True)

# get list of unique values for each label
gestures = np.unique(gestLabel)
positions = np.unique(posLabel)
trials = np.unique(trialLabel)

numGestures = len(gestures)
numPositions = len(positions)
numTrials = len(trials)

# get data size info
D = hv.shape[1] # hypervector dimension
numHV = 80 # number of examples per trial

# color palettes for plotting
gPalette = sns.color_palette('tab20', numGestures)
pPalette = sns.color_palette('tab20', numPositions)

In [6]:
numSplit = 20
skf = StratifiedKFold(n_splits=numSplit)

X = hv
y = gestLabel
c = posLabel
g = groupGP

splitIdx = 0
numIter = 100

acc = np.zeros((numSplit,numIter,numGestures,numPositions,2))
posWeights = np.ones((numSplit,numIter,numGestures,numPositions))


for trainIdx, testIdx in skf.split(X,g):
    print('Running cross validation split %d of %d...' % (splitIdx+1, numSplit))
    XTrain, XTest = X[trainIdx], X[testIdx]
    yTrain, yTest = y[trainIdx], y[testIdx]
    cTrain, cTest = c[trainIdx], c[testIdx]
    gTrain, gTest = g[trainIdx], g[testIdx]
    
    # get accuracy from equally weighting each arm position
    alpha = 3
    for n in range(numIter):
        weightMat = np.zeros(len(XTrain))
        for gest in gestures:
            for pos in positions:
                weightMat[(yTrain==gest) & (cTrain==pos)] = posWeights[splitIdx,n,gest,pos]
        weightMat = weightMat.reshape(-1,1)
        
        AM = centroids(XTrain*weightMat,label=yTrain)[1]
        
        for gest in gestures:
            for pos in positions:
                acc[splitIdx,n,gest,pos,0] = accuracy_score(classify(XTrain[(yTrain==gest) & (cTrain==pos)],AM,'hamming'),yTrain[(yTrain==gest) & (cTrain==pos)])
                acc[splitIdx,n,gest,pos,1] = accuracy_score(classify(XTest[(yTest==gest) & (cTest==pos)],AM,'hamming'),yTest[(yTest==gest) & (cTest==pos)])
        for gest in gestures:
            if n < numIter - 1:
                deltaWeights = softmax(1 - acc[splitIdx,n,gest,:,0])
                posWeights[splitIdx,n+1,gest,:] = posWeights[splitIdx,n,gest,:] + deltaWeights*alpha
        alpha *= 0.999
        print(np.mean(acc[0,n,:,:,0]))
    
    splitIdx += 1

Running cross validation split 1 of 20...
0.7599105937921726
0.77070681511471
0.7783400809716599
0.781798245614035
0.7853407557354924
0.7871963562753037
0.789389338731444
0.7910340755735491
0.7928053306342779
0.7930583670715249
0.7949983130904182
0.7959261133603237
0.7957995951417002
0.7955043859649122
0.7960526315789473
0.796221322537112
0.7962634952766531
0.7966008771929823
0.7968117408906881
0.7966430499325234
0.7970226045883939
0.7970226045883939
0.7971491228070173
0.7971069500674763
0.7974443319838056
0.7974443319838056
0.7974443319838056
0.7973599865047233
0.7974021592442645
0.7976973684210525
0.7976130229419702
0.7975708502024291
0.7976973684210525
0.7977817139001349
0.7976973684210527
0.7976551956815114
0.7976973684210525
0.7977395411605938
0.7977395411605938
0.7975708502024291
0.7978660593792173
0.7979925775978406
0.7979925775978406
0.7979925775978406
0.7979925775978406
0.7982034412955467
0.7981190958164643
0.7979082321187584
0.7974021592442645
0.797823886639676
0.797528677462

KeyboardInterrupt: 

In [ ]:
# nuXTrain = 20
# skf = StratifiedKFold(n_splits=numSplit)

# X = hv
# y = gestLabel
# c = posLabel
# g = groupGP

# splitIdx = 0
# numIter = 50

# acc = np.zeros((numSplit,numIter,numGestures,numPositions,2))
# posPrototypes = np.zeros((numSplit,numGestures,numPositions,D))
# posWeights = np.ones((numSplit,numIter,numGestures,numPositions,D))


# for trainIdx, testIdx in skf.split(X,g):
#     print('Running cross validation split %d of %d...' % (splitIdx+1, numSplit))
#     XTrain, XTest = X[trainIdx], X[testIdx]
#     yTrain, yTest = y[trainIdx], y[testIdx]
#     cTrain, cTest = c[trainIdx], c[testIdx]
#     gTrain, gTest = g[trainIdx], g[testIdx]
    
#     # gather separate prototype for each gesture performed in each arm position
    
#     for gest in gestures:
#         posPrototypes[splitIdx,gest,:,:] = centroids(XTrain[yTrain==gest],label=cTrain[yTrain==gest])[1]
    
#     # get accuracy from equally weighting each arm position
#     alpha = 0.1
#     for n in range(numIter):
#         AM = np.vstack([centroids(posPrototypes[splitIdx,g,:,:]*posWeights[splitIdx,n,g,:,:])[1][0] for g in gestures])
#         for gest in gestures:
#             for pos in positions:
#                 acc[splitIdx,n,gest,pos,0] = accuracy_score(classify(XTrain[(yTrain==gest) & (cTrain==pos)],AM,'hamming'),yTrain[(yTrain==gest) & (cTrain==pos)])
#                 acc[splitIdx,n,gest,pos,1] = accuracy_score(classify(XTest[(yTest==gest) & (cTest==pos)],AM,'hamming'),yTest[(yTest==gest) & (cTest==pos)])
#         for gest in gestures:
#             if n < numIter - 1:
#                 deltaWeights = softmax(1 - acc[splitIdx,n,gest,:,0]).reshape(-1,1)
#                 posWeights[splitIdx,n+1,gest,:,:] = posWeights[splitIdx,n,gest,:,:] + deltaWeights*alpha
#         alpha *= 0.99
#         print(np.mean(acc[0,n,:,:,0]))
    
#     splitIdx += 1